In [1]:
import sys
sys.path.append('../')

In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from constants import *
from map_utils import *
from eda_utils import *
from scipy.interpolate import griddata
import pickle as pkl
import datetime
import joblib

### Load Bihar GeoJSON file (for topology), and the modified CSV file that contains atleast 300 not nan values for each timestamp

In [3]:
bihar = gpd.read_file(f'{data_bihar}/bihar.json')
data_file = f'{data_bihar}/bihar_512_sensor_data_imputed.pkl'

In [4]:
# df = pd.DataFrame(columns={'timestamp': pd.Timestamp, 'latitude': np.float64, 'longitude': np.float64, 'rh': np.float32,\
#                            'temp': np.float64, 'pm25': np.float64})

cols = {'timestamp': pd.Timestamp, 'latitude': np.float64, 'longitude': np.float64, 'rh': np.float32,\
                           'temp': np.float64, 'pm25': np.float64}

df = pd.read_pickle(data_file)
# df = df.astype(cols)
# df['timestamp'] = df['timestamp'].values.astype(np.float64)
df['pm25'] = df['pm25'].astype(np.float64)
df.dtypes

timestamp    datetime64[ns]
block                object
district             object
latitude            float64
longitude           float64
rh                  float64
temp                float64
pm25                float64
dtype: object

### Data variable initialized below is a list, where each item of the list is a dictionary for a particular timestamp 

In [5]:
# data = []

# for i in range(0, len(df), 491):
#     row = {}
#     row['lat'] = impute_data[i:i+490, 1]
#     row['long'] = impute_data[i:i+490, 2]
#     row['pm25'] = preds[i:i+490]
#     data.append(row)

In [6]:
# df_ts = df.copy(deep=True)
# df_ts['timestamp'] = pd.to_datetime(df_ts['timestamp'], unit='ns')
# df_ts

In [7]:
min_lat, max_lat, min_long, max_long = coordinate_bounds(bihar)
print(min_lat, max_lat, min_long, max_long)

24.286327 27.521347 83.320238 88.29954611201047


#### data_ts_dict is a dictionary with mapping 'Timestamp' -> ['Latitude', 'Longitude', 'RH', 'Temp', 'PM25'] 

In [8]:
data_ts_dict = {timestamp: group for timestamp, group in df.groupby('timestamp')}
# data_ts_dict

In [9]:
data_ts_dict[pd.Timestamp('2023-07-21 18:00:00')]

,timestamp,block,district,latitude,longitude,rh,temp,pm25
5580104,2023-07-21 18:00:00,CHHAPRA,SARAN,25.783,84.746,95.325000,37.745000,17.050000
5848736,2023-07-21 18:00:00,SHEOHAR,SHEOHAR,26.516,85.285,63.835000,33.053333,14.316667
5919584,2023-07-21 18:00:00,BARAHAT,BANKA,24.877,87.007,63.659615,35.943846,6.346154
6084896,2023-07-21 18:00:00,LAUKAHI,MADHUBANI,26.469,86.562,67.781154,32.231410,12.119231
5270144,2023-07-21 18:00:00,DARBHANGA,DARBHANGA,26.157,85.898,68.191667,32.515833,10.983333
...,...,...,...,...,...,...,...,...
5340992,2023-07-21 18:00:00,BHABUA,KAIMUR,25.040,83.610,52.233333,35.599167,3.641667
5913680,2023-07-21 18:00:00,ITARHI,BUXAR,25.491,84.011,43.038333,40.296667,12.783333
5928440,2023-07-21 18:00:00,BELHAR,BANKA,24.918,86.599,52.505000,36.175000,3.650000
5984528,2023-07-21 18:00:00,PACHRUKHI,SIWAN,26.158,84.416,33.841520,42.125503,23.336683


#### data_dt_dict is a dictionary with mapping 'Date' -> List of data_ts_dict values with timestamp in that date

In [10]:
data_dt_dict = {}

for timestamp, values in data_ts_dict.items():
    date = timestamp.date()
    row = {timestamp: values}
    if date not in data_dt_dict:
        data_dt_dict[date] = []
    data_dt_dict[date].append(row)

In [11]:
# data_dt_dict

In [12]:
# len(data_dt_dict[datetime.date(2023, 8, 16)])

### Get Mask for points within Bihar region

In [13]:
pm25_values = []
grid_long, grid_lat = np.meshgrid(np.linspace(min_long, max_long, GRID_SIZE), np.linspace(min_lat, max_lat, GRID_SIZE))
# mask = get_indices(grid_long, grid_lat, bihar)
mask = np.loadtxt(f'{data_bihar}/mask.txt', dtype=bool, delimiter='\t')
# print(mask)

In [14]:
print(grid_long.shape, grid_lat.shape, mask.shape)

(1000, 1000) (1000, 1000) (1000, 1000)


#### Creating Average plots for each day

In [15]:
# for date, ts_dict in data_dt_dict.items():
#     dt = date.strftime('%Y-%m-%d')
#     tot_values = np.zeros((GRID_SIZE, GRID_SIZE))
#     for data in ts_dict:
#         for ts, val in data.items():
#             grid_values = griddata((val['latitude'], val['longitude']), val['pm25'], (grid_lat, grid_long), method='nearest')
#             tot_values = np.add(tot_values, grid_values)

#     day = date.strftime('%A')
#     dir = 'weekend' if day in ['Saturday', 'Sunday'] else 'weekday'

#     tot_values = tot_values / len(data)
#     create_plot(grid_long[mask], grid_lat[mask], tot_values[mask], bihar, f'{bihar_plot_dir}/{dir}/map_orig_{dt}', 'jpg')

#### EWMA parameters and calculation for each timestep, and creating the plots according to it

In [16]:
# ALPHA = 0.9
# ewma = pm25_values[0]
# ewma.shape

In [17]:
# for i, pm25 in enumerate(pm25_values):
#     ewma = np.add(ALPHA * pm25, (1 - ALPHA) * ewma)
        
# create_plot(grid_long[mask], grid_lat[mask], ewma[mask], bihar, f'map_orig_ewma', 'jpg')

## LCN

In [20]:
for date, ts_dict in data_dt_dict.items():
    dt = date.strftime('%Y-%m-%d')
    tot_values = np.zeros((GRID_SIZE, GRID_SIZE))
    
    for data in ts_dict:
        for ts, val in data.items():
            grid_values = griddata((val['latitude'], val['longitude']), val['pm25'], (grid_lat, grid_long), method='nearest')
            lcn_val = LCN(grid_long, grid_lat, grid_values)
            # mean, var = np.mean(lcn_val), np.var(lcn_val)
            # lcn_val = (lcn_val - mean) / np.sqrt(var)
            tot_values = np.add(tot_values, lcn_val)
            break
        break
    
    tot_values = tot_values / len(data)
    mean, var = np.mean(tot_values), np.var(tot_values)
    tot_values = (tot_values - mean) / np.sqrt(var)

    # day = date.strftime('%A')
    # dir = 'weekend' if day in ['Saturday', 'Sunday'] else 'weekday'

    # print(grid_long[mask])

    df = pd.DataFrame({'latitude': grid_lat[mask], 'longitude': grid_long[mask], 'pm25': tot_values[mask]})
    df.to_csv(f'{dt}.csv', index=False)
    create_plot(grid_long[mask], grid_lat[mask], tot_values[mask], bihar, f'{bihar_plot_dir}/map_LCN_{dt}', 'relative')
    break

In [ ]:
# ewma = LCN(grid_long, grid_lat, pm25_values[0])

In [ ]:
# for i, pm25 in enumerate(pm25_values):
#     smoothed_values = LCN(grid_long, grid_lat, pm25)
#     ewma = np.add(ALPHA * smoothed_values, (1 - ALPHA) * ewma)

# create_plot(grid_long[mask], grid_lat[mask], ewma[mask], bihar, f'map_LCN_ewma', 'jpg')

## Get Resolution for each point in the Image

We have favorable points as the number of points spread equally in the whole Bihar Region, and we know the total area of Bihar is 94,163 $km^{2}$ (publically available data), so we can find out the resolution for each point of image (i.e. area covered by each point) by simply dividing area by total number of favorable points

In [ ]:
print(np.sqrt(AREA_BIHAR/np.sum(mask)))

Hence, each point covers approximately $529.56 \; m * 529.56 \; m$ area